In [1]:
import sys
sys.path.insert(1, '/home/zhouc1/notebooks/definexpfam/')

In [2]:
import os
from copy import deepcopy
import numpy as np
import pandas as pd
from data_median_dist import *
from base_density import *
from basis_function import *

from scorematching_finexpfam import *
from evaluate_scorematching_loss import *
from negloglik_finexpfam import *
from unnormalized_density import *
from plot_density_1d import *

import scipy 

from datetime import datetime 

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
%matplotlib inline 

import matplotlib.gridspec as gridspec

In [ ]:
def scorematching_finexpfam_coef11(data, basis_function, base_density):

    """
    Returns the solution to minimizing the score matching loss function
    in finite-dimensional exponential family.

    Parameters
    ----------
    data : numpy.ndarray
        The array of observations whose density function is to be estimated.

    basis_function : kernel_function object
        The basis function of the canonical statistic used to estimate the probability density function.
        __type__ must be 'basis_function'.

    base_density : base_density object
        The base density function used to estimate the probability density function.
        __type__ must be 'base_density'.

    Returns
    -------
    numpy.ndarray
        An array of coefficients for the natural parameter in the score matching density estimate.

    """

    N, d = data.shape
    n_basis = basis_function.landmarks.shape[0]

    # compute DT(X_i) DT(X_i)^\top
    DT = basis_function.basisfunction_deriv1(data)
    dt_prod_term = sum([np.matmul(DT[:, i].reshape(n_basis, d),
                                  DT[:, i].reshape(n_basis, d).T) for i in range(N)])

    # compute the matrix G, which involves the second derivative
    matG = basis_function.basisfunction_deriv2(data)
    sum_matG = np.sum(sum([matG[:, i].reshape(n_basis, d) for i in range(N)]), axis=1, keepdims=True)

    # compute DT and grad of log mu
    # compute the gradient of log mu at data
    # each row corresponds to one data point
    grad_logmu = np.array([base_density.logbaseden_deriv1(data, j).flatten() for j in range(d)]).T
    dt_baseden_term = sum([np.matmul(DT[:, i].reshape(n_basis, d), grad_logmu[[i]].T) for i in range(N)])

    b_term = sum_matG + dt_baseden_term

    coef = -np.linalg.lstsq(dt_prod_term, b_term, rcond=None)[0]

    return dt_prod_term, b_term, coef

In [3]:
# os.chdir('/Users/chenxizhou/Dropbox/Research_Density_Estimation/data_analysis/old_faithful_geyser')
os.chdir('/home/zhouc1/notebooks/definexpfam_results/iris')
true_data = pd.read_csv('iris.csv')# .astype(np.float64)
var_name = 'Petal_Width'
df = deepcopy(true_data[var_name]).to_numpy().reshape(-1, 1)
xlimit = (0., 4.)
ylimit = (-0.005, 1.42)

# sepal_length = deepcopy(true_data['Sepal_Length']).to_numpy().reshape(-1, 1) # (2., 10.), (-0.005, 0.99)
# sepal_width = deepcopy(true_data['Sepal_Width']).to_numpy().reshape(-1, 1) # (0., 6.), (-0.005, 0.99)
# petal_length = deepcopy(true_data['Petal_Length']).to_numpy().reshape(-1, 1) # (0.5, 9.5), (-0.005, 1.42)
# petal_width = deepcopy(true_data['Petal_Width']).to_numpy().reshape(-1, 1) # (0., 4.), (-0.005, 1.42)

In [ ]:
# # use Freedman-Diaconis rule to determine the binwidth 
# ax = pd.Series(df.flatten()).hist(grid = False, figsize = (10, 10), bins = 'fd', density = True) 
# ax.set_ylim(ylimit)
# ax.set_xlim(xlimit)
# ax.set_xlabel(var_name)
# # ax.figure.savefig('hist_eruptions.pdf')

In [4]:
np.unique(df.flatten())

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6,
       1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5])

In [5]:
# ------------------------------------------------------------------------
# # add an data point 
# df = np.vstack([df, np.array([7.287])])

# df[df == 2.0] = 1.885

landmarks = np.unique(deepcopy(true_data[var_name])).reshape(-1, 1)
# landmarks = np.linspace(0.1, 6.9, 10).reshape(-1, 1)

In [ ]:
pd.Series(df.flatten()).describe()

In [ ]:
landmarks.flatten()
# np.unique(data_waiting.flatten())

In [ ]:
# med_dist = data_median_dist(df.reshape(-1, 1))
# med_dist

In [ ]:
bw = 1.0
basis_function = GaussianBasisFunction(
    landmarks = landmarks, 
    bw = bw)
base_density = BasedenGamma(df, 3.5170581590028753, 1.068506641091609)
base_density.a, base_density.scale

In [ ]:
# sepal_length, 49.351702439900336, 0.11818134011138696
# sepal_width, 49.651897882066784, 0.06157535690972203
# petal_length, 3.5170581590028753, 1.068506641091609
# petal_width, 1.5578209222227564, 0.7698788199750717

In [ ]:
coef_sm = scorematching_finexpfam_coef(
    data = df, 
    basis_function = basis_function, 
    base_density = base_density)

In [ ]:
file_name = f'add{np.max(df)}_coef_scorematching_bw={basis_function.bw}_unique.npy'
# f'shift2.0to{np.min(df)}_coef_scorematching_bw={basis_function.bw}_unique.npy'
# f'original_coef_scorematching_bw={basis_function.bw}_unique.npy'

np.save(f'{var_name}/{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name, 
       coef_sm)
# # np.save(f'bandwidth_effect/{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name, 
# #        coef_sm)

In [ ]:
plot_domain = xlimit
plot_pts_cnt = 4000
x0 = np.linspace(plot_domain[0], plot_domain[1], plot_pts_cnt).reshape(-1, 1)

basis_mat = basis_function.basisfunction_eval(x0)
y0 = np.matmul(basis_mat.T, coef_sm).flatten()
mu = base_density.baseden_eval(x0).flatten()

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(x0, np.log(mu) + y0, 'b-')
# plt.axvline(108., 0, 1)
#plt.ylim((5.6e5, 5.7e5))

In [ ]:
np.max(y0)

In [ ]:
sub_const = 3400.
def density_eval_1d_cus(basis_function, base_density, coef):

    landmarks = basis_function.landmarks
    n_obs = landmarks.shape[0]
    
    def den(x): 

        den_val = (base_density.baseden_eval_1d(x) * 
                   np.exp(- sub_const + np.sum([coef[i] * basis_function.basis_x_1d(landmarks[i])(x)
                          for i in range(n_obs)])))
        
        return den_val

    return den

In [ ]:
int_result, _ = scipy.integrate.quad(
    density_eval_1d_cus(
        basis_function = basis_function, 
        base_density = base_density, 
        coef = coef_sm), 
    a = 0., 
    b = np.inf, 
limit=1000)
int_result

In [ ]:
den_vals = mu * np.exp(y0 - sub_const) / int_result 
plt.figure(figsize = (10, 10))
plt.plot(x0, den_vals, 'b-')
plt.xlim(plot_domain) # xlimit)
#plt.ylim((-0.01, 20))
plt.ylim(ylimit)
#plt.axvline(115., 0, 1)
# plt.vlines(x = 75., ymin = -1, ymax = 1)

In [ ]:
file_name = f'add{np.max(df)}_denvals_scorematching_bw={basis_function.bw}_unique.npy'
# f'shift2.0to{np.min(df)}_denvals_scorematching_bw={basis_function.bw}_unique.npy'
# f'original_denvals_scorematching_bw={basis_function.bw}_unique.npy'

np.save(f'{var_name}/{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name, 
        den_vals)
# np.save(f'{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name, 
#         den_vals)

In [ ]:
# bw_list = np.linspace(14.5513, 14.5514, 100)
outlier_list = np.arange(2010, 2020) / 1000.
outlier_list

In [ ]:
loss_list = []

In [ ]:
plot_domain = xlimit
plot_pts_cnt = 4000
x0 = np.linspace(plot_domain[0], plot_domain[1], plot_pts_cnt).reshape(-1, 1)

bw = 0.6
var_name = 'Sepal_Width'

# for i in range(len(bw_list)): 
for i in range(len(outlier_list)): 
    print('*' * 50)
    # bw = bw_list[i]
    # print(f'i = {i}, bw = {bw}')
    
    outlier = outlier_list[i]
    print(f'i = {i}, outlier = {outlier}')
    
    data_frame = deepcopy(true_data[var_name].to_numpy()) # .reshape(-1, 1)
    data_frame = np.append(data_frame, outlier)
    print(f'max = {np.max(data_frame)}')
    data_frame = data_frame.reshape(-1, 1)
    print(pd.Series(data_frame.flatten()).describe())
    
    base_density = BasedenGamma(data_frame, 1.5578209222227564, 0.7698788199750717)
    basis_function = GaussianBasisFunction(
        landmarks = landmarks, # data_eruptions, 
        bw = bw)
    
    coef_sm = scorematching_finexpfam_coef(
        data = data_frame, 
        basis_function = basis_function, 
        base_density = base_density)
    
    file_name = f'add{np.max(data_frame)}_coef_scorematching_bw={basis_function.bw}_unique.npy'
# f'shift2.0to{np.min(df)}_coef_scorematching_bw={basis_function.bw}_unique.npy'
# f'original_coef_scorematching_bw={basis_function.bw}_unique.npy'

    np.save(f'{var_name}/{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name, 
       coef_sm)
    
    basis_mat = basis_function.basisfunction_eval(x0)
    mu = base_density.baseden_eval(x0).flatten()
    y0 = np.matmul(basis_mat.T, coef_sm).flatten()
    
    sub_const = 20000.

    def density_eval_1d_cus(basis_function, base_density, coef):

        landmarks = basis_function.landmarks
        n_obs = landmarks.shape[0]

        def den(x): 

            den_val = (base_density.baseden_eval_1d(x) * 
                       np.exp(- sub_const + np.sum([coef[i] * basis_function.basis_x_1d(landmarks[i])(x)
                              for i in range(n_obs)])))

            return den_val

        return den


    int_result, _ = scipy.integrate.quad(
    density_eval_1d_cus(
        basis_function = basis_function, 
        base_density = base_density, 
        coef = coef_sm), 
    a = 0., 
    b = np.inf, 
    limit=1000)
    
    plt.figure(figsize=(7, 7))
    den_vals = mu * np.exp(y0 - sub_const) / int_result 
    plt.plot(x0, den_vals, 'b-')
    plt.xlim(plot_domain) # xlimit)
    #plt.ylim((-0.01, 20))
    plt.ylim(ylimit)
    plt.vlines(outlier, 0, 1)
    plt.show()
    
    file_name = f'shift2.0to{np.min(data_frame)}_denvals_scorematching_bw={basis_function.bw}_unique.npy'
    # f'shift2.0to{np.min(df)}_denvals_scorematching_bw={basis_function.bw}_unique.npy'
    # f'original_denvals_scorematching_bw={basis_function.bw}_unique.npy'

    np.save(f'{var_name}/{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name, 
        den_vals)
    
#     smloss = evaluate_scorematching_loss(data_frame, basis_function, base_density, coef_sm)
#     loss_list.append(smloss)
#     print(f'Score Matching Loss = {smloss[2]:.5f}, Squared deriv1 = {smloss[0]:.5f}, deriv2 = {smloss[1]:.5f}.')
    

In [ ]:
# log_denvals = np.log(mu) + y0
# plt.figure(figsize=(10, 10))
# plt.plot(x0, log_denvals, 'b-')
# plt.axvline(108., 0, 1)
# plt.ylim((0.79e5, 0.81e5))

In [ ]:
file_name = f'shift108to99_logdenvals_scorematching_bw={basis_function.bw}_35_110_16.npy'
np.save(f'dataloc_effect/{basis_function.basisfunction_name}_basis_function/' + file_name, 
        log_denvals)

In [ ]:
# basis_function.basisfunction_name, basis_function.bw

In [ ]:
file_name = f'original_denvals_scorematching_bw=0.3.npy'
yy0 = np.load(f'{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name)

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(x0, yy0/1e105, 'b-')
plt.xlim(xlimit)
plt.ylim(ylimit)
# plt.vlines(x = 75., ymin = -1, ymax = 1)

In [ ]:
file_name = f'original_denvals_scorematching_bw=0.3.npy'
np.save(f'{basis_function.basisfunction_name}_basis_function/scorematching/' + file_name, yy0/1e105)

In [ ]:
batchmc_params = batch_montecarlo_params(
    mc_batch_size = 1000, 
    mc_tol = 1e-2)

nll_algo_params = negloglik_optalgoparams(
    start_pt = np.zeros((data_eruptions.shape[0], 1)), 
    step_size = 0.03, 
    max_iter = 100, 
    rel_tol = 2e-5)

In [ ]:
now1 = datetime.now()

coef_lik = negloglik_finexpfam_coef(
    data = data_eruptions, 
    basis_function = basis_function, 
    base_density = base_density, 
    optalgo_params = nll_algo_params, 
    batchmc_params = batchmc_params,
    batch_mc = True, 
    batch_mc_se = False, 
    print_error = True)
now2 = datetime.now()
print(now1, now2)
print(now2 - now1)

In [ ]:
file_name = f'original_coef_likelihood_bw={basis_function.bw}.npy'
np.save(f'{basis_function.basisfunction_name}_basis_function/likelihood/' + file_name, 
        coef_lik)

In [ ]:
plot_domain = xlimit
plot_pts_cnt = 4000
x0 = np.linspace(plot_domain[0], plot_domain[1], plot_pts_cnt).reshape(-1, 1)

In [ ]:
basis_mat = basis_function.basisfunction_eval(x0)
y0 = np.matmul(basis_mat.T, coef_lik).flatten()
mu = base_density.baseden_eval(x0).flatten()

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(x0, np.log(mu) + y0, 'b-')
#plt.ylim((4e6, 4.3e6))

In [ ]:
file_name = f'original_coef_likelihood_bw={basis_function.bw}.npy'
coef_lik = np.load(f'{basis_function.basisfunction_name}_basis_function/likelihood/' + file_name)

In [ ]:
unnorm = UnnormalizedDensityFinExpFam(
    data = data_eruptions, 
    basis_function = basis_function, 
    base_density = base_density, 
    coef = coef_lik)

In [ ]:
unnorm_fun = unnorm.density_eval
unnorm_fun_int = unnorm.density_eval_1d
x0_cand = np.linspace(xlimit[0], xlimit[1], num=4000).reshape(-1, 1)
plot_val = unnorm_fun(x0_cand)

In [ ]:
norm_const, _ = integrate.nquad(unnorm_fun_int, base_density.domain, opts={'limit': 100})
norm_const

In [ ]:
file_name = f'original_coef_likelihood_bw=0.4.npy'
coef_lik = np.load(f'{basis_function.basisfunction_name}_basis_function/likelihood/' + file_name)

In [ ]:
plot_params = plot_density_1d_params(x_limit = xlimit, y_limit = ylimit, plot_pts_cnt=4000)

In [ ]:
denvals = plot_density_1d(
    data = data_eruptions, 
    basis_function = basis_function, 
    base_density = base_density, 
    coef = coef_lik, 
    normalizing = True, 
    method = 'likelihood', 
    x_label = 'waiting',
    plot_kwargs = plot_params, 
    save_plot = False, 
    save_dir = None, 
    save_filename = None)

In [ ]:
file_name = f'original_denvals_likelihood_bw={basis_function.bw}.npy'
np.save(f'{basis_function.basisfunction_name}_basis_function/likelihood/' + file_name, 
        denvals['den_vals'])